In [1]:
from transformers import BertConfig, BertForMaskedLM, BertTokenizer
from transformers import PreTrainedTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
from torch.nn.functional import pad
from datasets import load_dataset
from tqdm.notebook import tqdm
from itertools import cycle
import wandb

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
wandb.login(key='56853ea1087c0d089c005f3f7aede52740245615')

device

2024-09-11 14:54:55.435305: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mb_lims (lims-ai). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/mburtsev/.netrc


device(type='cuda', index=3)

# Create a custom tokenizer for CA

In [2]:
class SimpleTokenizer:
    def __init__(self):
        self.token_to_id = {'0': 0, '1': 1, '[SEP]': 2, '[PAD]': 3, '[T]': 4} # [T] - thought
        self.id_to_token = {v: k for k, v in self.token_to_id.items()}

    def encode(self, text):
        return [self.token_to_id.get(token, self.token_to_id['[PAD]']) for token in text]

    def decode(self, tokens):
        return ''.join([self.id_to_token.get(token, '[PAD]') for token in tokens])
        #return ''.join([self.id_to_token[token] for token in tokens])

    def __len__(self):
        return len(self.token_to_id)

tokenizer = SimpleTokenizer()

# Define CA dataset

In [3]:
class CellularAutomataDataset(Dataset):
    
    def __init__(self, hf_dataset, steps, horizon=1, pad_length=512):
        self.hf_dataset = hf_dataset
        self.pad_length = pad_length
        self.steps = steps
        self.horizon = horizon

    def __len__(self):
        return len(self.hf_dataset)

    def __getitem__(self, idx):
        row = self.hf_dataset[idx]

        # Assuming the first element 'rule' is not used for model input
        # Concatenate all states except the last one for input
        if (self.steps > (len(row) - 1)):
            self.steps = len(row) - 1
        
        input_ids = []
        for t in range(self.steps):  # Exclude the last state and the 'rule' row
            input_ids.extend(tokenizer.encode(row[f't={t}']))
            input_ids.append(2)  # Append [SEP] token after each state

        # Use the last state as target and calculate its length
        target_ids = []
        for t in range(self.horizon):
            last_state_key = f't={self.steps + t}'
            target_ids.extend(tokenizer.encode(row[last_state_key]))
            target_ids.append(2)
        
        target_ids.extend(tokenizer.encode(row['rule']))
        
        orbit_len = len(input_ids)
        target_len = len(target_ids)
        # Padding input and target sequences to the fixed length
        input_ids = pad(torch.tensor(input_ids), (0, self.pad_length - orbit_len), value=3)
        target_ids = pad(torch.tensor(target_ids), (orbit_len, self.pad_length - orbit_len - target_len), value=3)

        # Create a tensor of -100 and replace the last part with target_ids
       # target_labels = torch.full((self.pad_length,), -100)
        #target_labels[:state_length] = target_ids[:state_length]

        return input_ids, target_ids #target_labels


# Configuration for the BERT model

In [4]:
hidden_size=512
num_hidden_layers=4
num_attention_heads=8

config = BertConfig(
    vocab_size=len(tokenizer),
    hidden_size=hidden_size,
    num_hidden_layers=num_hidden_layers,
    num_attention_heads=num_attention_heads,
    #intermediate_size=3072
)
model_name = 'BERT'
model = BertForMaskedLM(config=config)

# Load dataset from Hugging Face repository

In [5]:
seed = 0
batch_size = 400
torch.manual_seed(seed)
steps = 10
state_len = 21 
look_ahead = 1
orbit_len = state_len * steps
seq_len = orbit_len + state_len * look_ahead + 32

dataset_name = "1dCA_r2s20T20"
hf_dataset = load_dataset("mbur/"+dataset_name)['train']
dataset = CellularAutomataDataset(hf_dataset, steps, look_ahead, pad_length=seq_len)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

hf_dataset = load_dataset("mbur/"+dataset_name)['validation']
val_dataset = CellularAutomataDataset(hf_dataset, steps, look_ahead, pad_length=seq_len)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

hf_dataset = load_dataset("mbur/"+dataset_name)['test']
test_dataset = CellularAutomataDataset(hf_dataset, steps, look_ahead, pad_length=seq_len)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

validation_size = 1000  # Number of samples for validation
small_val_dataset, _ = random_split(val_dataset, [validation_size, len(val_dataset) - validation_size])

small_val_dataloader = DataLoader(small_val_dataset, batch_size=batch_size, shuffle=True)

Using custom data configuration mbur--1dCA_r2s20T20-062a8ac2d777e207
Found cached dataset json (/home/mburtsev/.cache/huggingface/datasets/mbur___json/mbur--1dCA_r2s20T20-062a8ac2d777e207/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/3 [00:00<?, ?it/s]

Using custom data configuration mbur--1dCA_r2s20T20-062a8ac2d777e207
Found cached dataset json (/home/mburtsev/.cache/huggingface/datasets/mbur___json/mbur--1dCA_r2s20T20-062a8ac2d777e207/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/3 [00:00<?, ?it/s]

Using custom data configuration mbur--1dCA_r2s20T20-062a8ac2d777e207
Found cached dataset json (/home/mburtsev/.cache/huggingface/datasets/mbur___json/mbur--1dCA_r2s20T20-062a8ac2d777e207/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
# Define the ID of the sample you want to inspect
sample_id = 1  # Replace with the ID of the sample you're interested in

# Fetch the specific sample from the dataset
sample_input_ids, sample_target_ids = dataset[sample_id]
sample_input_ids, sample_target_ids = sample_input_ids.unsqueeze(0).to(device), sample_target_ids.unsqueeze(0).to(device)


# Decode the tokens to strings for readability
decoded_input = tokenizer.decode(sample_input_ids[0].tolist())
decoded_target = tokenizer.decode(sample_target_ids[0].tolist())

# Print input, target, prediction, and logits
print("inputs:",len(sample_input_ids),sample_input_ids.tolist())
print("targets:",len(sample_target_ids),sample_target_ids.tolist())
print("Input:", len(sample_input_ids[0].tolist()),decoded_input)
print("Target:", len(sample_target_ids[0].tolist()), decoded_target)

inputs: 1 [[1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 2, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 2, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 2, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 2, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 2, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 2, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]]
targets: 1 [[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

# Evaluation

In [7]:
def evaluate_model(model, val_dataloader, device):
    model.eval()  # Set the model to evaluation mode
    total, correct = 0, 0
    correct_r, total_r = 0, 0
    val_loop = tqdm(val_dataloader, leave=False, desc="Validation")

    with torch.no_grad():
        for input_ids, target_ids in val_loop:
            input_ids, target_ids = input_ids.to(device), target_ids.to(device)

            outputs = model(input_ids)
            predictions = torch.argmax(outputs.logits, dim=-1)

            # Create a mask where target_ids is not -100 (ignoring padding)
            valid_mask = target_ids != 3

            # Apply the mask to filter out -100 values
            valid_predictions = predictions[valid_mask]
            valid_targets = target_ids[valid_mask]
            # for state
            correct += (valid_predictions[:-32] == valid_targets[:-32]).sum().item() 
            total += valid_targets[:-32].numel()
            # for rule
            correct_r += (valid_predictions[-32:] == valid_targets[-32:]).sum().item() 
            total_r += valid_targets[-32:].numel()

    accuracy = correct / total if total > 0 else 0
    accuracy_rule = correct_r / total_r if total > 0 else 0
    
    return accuracy, accuracy_rule

# Training loop

In [8]:
model.to(device)

lr=1e-4 # learning rate

num_steps = 25_000  # Define the total number of training steps
evaluate_every = 100  # Frequency of evaluation
wndb_log = 10

run_cfg = {
    'dataset': dataset_name,
    'model': model_name,
    'input_size': 512,
    'hidden_size': hidden_size,
    'num_hidden_layers': num_hidden_layers,
    'num_attention_heads': num_attention_heads,
    'batch_size': batch_size,
    'learning rate': lr,
    'seed': seed
}
run_name = dataset_name+'i'+str(steps)+'f'+str(look_ahead)+'H2shrt_'+model_name+'_'+str(num_hidden_layers)+'L'+str(num_attention_heads)+'H'+str(hidden_size)+'HD'+'_bs'+str(batch_size)+'lr'+str(lr)
run_name = run_name + str(num_steps) + 'Kgc_v' + str(seed)
run = wandb.init(
    project="1dCA_ai4math24",
    name = run_name,
    config=run_cfg
)

optimizer = torch.optim.AdamW(model.parameters(), lr)

losses = []

model.train()

loop = tqdm(range(num_steps), desc='Training')
data_iter = cycle(dataloader)  # Create an infinite loop of dataloader

for step in loop:
    
    input_ids, target_ids = next(data_iter)
    input_ids, target_ids = input_ids.to(device), target_ids.to(device)

    outputs = model(input_ids, labels=target_ids)
    loss = outputs.loss

    optimizer.zero_grad()
    loss.backward()
    total_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
    optimizer.step()
    current_loss = loss.item()
    loop.set_description(f"Step {step + 1}/{num_steps} - Loss: {current_loss:.4f}")
    loop.set_postfix(loss=current_loss)

    if step % evaluate_every == 0 and step != 0:
        val_accuracy, rule_accuracy = evaluate_model(model, small_val_dataloader, device)
        print(f"\nEvaluation at Step {step}: Accuracy = {val_accuracy:.4f}, Rule acc:{rule_accuracy:.4f} ", "loss:", loss.item())
        wandb.log({'step': step, 'val acc': val_accuracy})

    if step % wndb_log == 0:
        losses.append(loss.detach().item())      
        wandb.log({'step': step, 'loss': current_loss})

loop.close()
wandb.finish()

Training:   0%|          | 0/25000 [00:00<?, ?it/s]

Validation:   0%|          | 0/3 [00:00<?, ?it/s]


Evaluation at Step 100: Accuracy = 0.5867, Rule acc:0.5729  loss: 0.13389559090137482


Validation:   0%|          | 0/3 [00:00<?, ?it/s]


Evaluation at Step 200: Accuracy = 0.5889, Rule acc:0.5417  loss: 0.1328190416097641


...

Validation:   0%|          | 0/3 [00:00<?, ?it/s]


Evaluation at Step 23100: Accuracy = 0.9609, Rule acc:0.9167  loss: 0.010794112458825111


Validation:   0%|          | 0/3 [00:00<?, ?it/s]


Evaluation at Step 23200: Accuracy = 0.9606, Rule acc:0.9583  loss: 0.012196863070130348


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [22]:
# Define the checkpoint content
checkpoint = {
    'model_state_dict': model.state_dict(),            # Model's state dictionary
    'optimizer_state_dict': optimizer.state_dict(),        # Optimizer's state dictionary
    'step': step,                                      # Current training step
    'losses': losses,                                  # Loss history
    'config': run_cfg,
    'seed': seed
}

run_name = dataset_name+'i'+str(steps)+'f'+str(look_ahead)+'H2shrt_OR_'+model_name+'_'+str(num_hidden_layers)+'L'+str(num_attention_heads)+'H'+str(hidden_size)+'HD'+'_bs'+str(batch_size)+'lr'+str(lr)
run_name = run_name + str(num_steps) + 'Kgc_v' + str(seed)

# Define a checkpoint file path
checkpoint_path = run_name + ".pth"

# Save the checkpoint
torch.save(checkpoint, checkpoint_path)

In [10]:
# Define the ID of the sample you want to inspect
sample_id = 11  # Replace with the ID of the sample you're interested in

# Fetch the specific sample from the dataset
sample_input_ids, sample_target_ids = dataset[sample_id]
sample_input_ids, sample_target_ids = sample_input_ids.unsqueeze(0).to(device), sample_target_ids.unsqueeze(0).to(device)

# Set the model to evaluation mode and get output
model.eval()
with torch.no_grad():
    output = model(sample_input_ids)

# Decode the tokens to strings for readability
decoded_input = tokenizer.decode(sample_input_ids[0].tolist())
decoded_target = tokenizer.decode(sample_target_ids[0].tolist())
decoded_prediction = tokenizer.decode(torch.argmax(output.logits, dim=-1)[0].tolist())

# Print input, target, prediction, and logits
print("inputs:",len(sample_input_ids),sample_input_ids)
print("targets:",len(sample_target_ids),sample_target_ids)
print(len(output),output)
print("Input:", len(sample_input_ids[0].tolist()),decoded_input)
print("Target:", len(sample_target_ids[0].tolist()), decoded_target)
print("Prediction:", decoded_prediction)
print("Logits:", len(output.logits[0]), output.logits[0])

# Optionally, you can convert logits to probabilities for interpretation
probabilities = torch.softmax(output.logits[0], dim=-1)
print("Probabilities:", probabilities)


inputs: 1 tensor([[1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 2, 1, 0, 1,
         1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 2, 0, 0, 1, 0, 0, 0,
         1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 2, 0, 1, 1, 1, 0, 1, 1, 0, 0,
         0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 2, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
         0, 0, 0, 0, 1, 1, 0, 0, 2, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0,
         1, 1, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
         0, 1, 2, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 2,
         1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 2, 1, 1, 0,
         1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 2, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]],
       device='cuda:3')
targets: 1 tensor([[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [19]:
def evaluate_model_LH(model, val_dataloader, device, horizon):
    model.eval()  # Set the model to evaluation mode
    val_loop = tqdm(val_dataloader, leave=False, desc="Validation")
    correct = [0] * horizon
    total = [0] * horizon
    accuracy = [0] * horizon
    correct_r = 0
    total_r = 0

    with torch.no_grad():
        for input_ids, target_ids in val_loop:
            input_ids, target_ids = input_ids.to(device), target_ids.to(device)

            outputs = model(input_ids)
            predictions = torch.argmax(outputs.logits, dim=-1)
            
            for t in range(horizon):

                # Create a mask where target_ids is not 3 (ignoring padding)
                #valid_mask = target_ids[:, t] != 3

                # Apply the mask to filter out -100 values
                start_idx  = orbit_len + t * state_len
                valid_predictions = predictions[:, start_idx : start_idx + state_len]
                valid_targets = target_ids[:, start_idx : start_idx + state_len]
            

                correct[t] += (valid_predictions == valid_targets).sum().item()
                total[t] += len(valid_targets)
                
            # for rule
            valid_predictions = predictions[:, start_idx + state_len : start_idx + state_len + 32]
            valid_targets = target_ids[:, start_idx + state_len : start_idx + state_len + 32]
            
            correct_r += (valid_predictions == valid_targets).sum().item()
            total_r += len(valid_targets)

    for t in range(horizon):
        accuracy[t] = correct[t] / total[t] / state_len  if total[t] > 0 else 0
        
    accuracy_rule = correct_r / total_r / 32 if total_r > 0 else 0
        
    return accuracy, correct, total, accuracy_rule

In [20]:
val_accuracy = evaluate_model_LH(model, val_dataloader, device,look_ahead)
print(val_accuracy)
#print(f"\nEvaluation at Step {step}: Validation accuracy = {val_accuracy:.10f}", "loss:", loss.item())

Validation:   0%|          | 0/125 [00:00<?, ?it/s]

([0.9927457142857143], [1042383], [50000], 0.944338125)


In [21]:
test_accuracy = evaluate_model_LH(model, test_dataloader, device,look_ahead)
print(test_accuracy)
#print(f"\nEvaluation at Step {step}: Test Accuracy = {test_accuracy:.10f}", "loss:", loss.item())

Validation:   0%|          | 0/250 [00:00<?, ?it/s]

([0.992742380952381], [2084759], [100000], 0.94414875)


In [ ]:
Evaluation at Step 49999: Accuracy = 0.999925 loss: 0.0001053252344718203

In [ ]:
orbit_len = state_len * steps

In [ ]:
0.9765709523809524